In [56]:
from bitarray import bitarray
import struct

In [57]:
def openFile(path):
    """Otwiera plik i zwraca jego zawartosc"""
    with open(path, "rb") as f:
        return f.read()

def decToBin(n):
    """Zamienia liczbe dziesietna na binarna"""
    if n == 0:
        return "0"
    elif n == 1:
        return "1"
    else:
        return decToBin(n // 2) + str(n % 2)

def toBits(n):
    """Zamienia liczbe na binarna i dodaje zera na poczatek"""
    bits = decToBin(n)
    #kod ma 6 bitow
    while len(bits) < 6:
        bits = "0" + bits
    return bits

In [58]:
def encode(text):
    """Kodowanie LZW"""
    dictionary = {bytes([i]): i for i in range(256)}
    dict_size = 256

    p = b""          # bieżący „pattern”
    encoded = []     # zakodowane dane

    for byte in text:
        c = bytes([byte])
        pc = p + c
        if pc in dictionary: # jeśli jest w słowniku
            p = pc
        else:
            # jeśli nie ma
            encoded.append(dictionary[p])
            dictionary[pc] = dict_size
            dict_size += 1
            p = c

    #ostatni pattern
    if p:
        encoded.append(dictionary[p])

    return encoded

In [59]:
def decode(text):
    """Dekodowanie LZW"""
    dictionary = {i: bytes([i]) for i in range(256)}
    dict_size = 256

    # Pierwszy kod
    iter_codes = iter(text)
    first_code = next(iter_codes)
    w = dictionary[first_code]
    result = bytearray(w)

    # Reszta
    for k in iter_codes:
        if k in dictionary:
            entry = dictionary[k]
        elif k == dict_size:
            # przypadek specialny LZW
            entry = w + w[:1]
        else:
            raise ValueError("Nieprawidłowy kod w dekodowaniu LZW")
        
        result.extend(entry)
        # dodaj nowy wpis do słownika
        dictionary[dict_size] = w + entry[:1]
        dict_size += 1

        w = entry

    return bytes(result)

In [60]:
def save(text, path):
    """Zapisuje zakodowane dane do pliku"""
    with open(path, 'wb') as f:
        for code in text:
            f.write(struct.pack('>I', code))


def load(path):
    """Wczytuje plik i zwraca jego zawartosc"""
    codes = []
    with open(path, 'rb') as f:
        while True:
            bytes_read = f.read(4)
            if not bytes_read:
                break
            codes.append(struct.unpack('>I', bytes_read)[0])
    return codes

In [61]:
path1 = "dane/lena.bmp"
path2 = "dane/norm_wiki_sample.txt"
path3 = "dane/wiki_sample.txt"
lena = openFile(path1)
normWikiSample = openFile(path2)
wikiSample = openFile(path3)
# print(lena[:1000])
# print(normWikiSample[:1000])
# print(wikiSample[:1000])

In [62]:
encodedLena = encode(lena)
decodedLena = decode(encodedLena)
encodedNormWikiSample = encode(normWikiSample)
decodedNormWikiSample = decode(encodedNormWikiSample)
encodedWikiSample = encode(wikiSample)
decodedWikiSample = decode(encodedWikiSample)
print("Czy zakodowany tekst jest taki sam jak oryginalny?", decodedLena == lena)
print("Czy zakodowany tekst jest taki sam jak oryginalny?", decodedNormWikiSample == normWikiSample)
print("Czy zakodowany tekst jest taki sam jak oryginalny?", decodedWikiSample == wikiSample)

Czy zakodowany tekst jest taki sam jak oryginalny? True
Czy zakodowany tekst jest taki sam jak oryginalny? True
Czy zakodowany tekst jest taki sam jak oryginalny? True


In [63]:
print("Lena       ", lena[:100])
print("decodedLena", decodedLena[:100])
print("normWikiSample       ", normWikiSample[:100])
print("decodedNormWikiSample", decodedNormWikiSample[:100])
print("wikiSample       ", wikiSample[:100])
print("decodedWikiSample", decodedWikiSample[:100])

Lena        b'BMJ\xdb\xaf\x00\x00\x00\x00\x00\x8a\x00\x00\x00|\x00\x00\x00\xa8\x07\x00\x00\xa8\x07\x00\x00\x01\x00\x18\x00\x00\x00\x00\x00\xc0\xda\xaf\x00\x12\x0b\x00\x00\x12\x0b\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xff\x00\x00\xff\x00\x00\xff\x00\x00\x00\x00\x00\x00\xffBGRs\x80\xc2\xf5(`\xb8\x1e\x15 \x85\xeb\x01@33\x13\x80ff&@ff\x06\xa0\x99'
decodedLena b'BMJ\xdb\xaf\x00\x00\x00\x00\x00\x8a\x00\x00\x00|\x00\x00\x00\xa8\x07\x00\x00\xa8\x07\x00\x00\x01\x00\x18\x00\x00\x00\x00\x00\xc0\xda\xaf\x00\x12\x0b\x00\x00\x12\x0b\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xff\x00\x00\xff\x00\x00\xff\x00\x00\x00\x00\x00\x00\xffBGRs\x80\xc2\xf5(`\xb8\x1e\x15 \x85\xeb\x01@33\x13\x80ff&@ff\x06\xa0\x99'
normWikiSample        b' albert of prussia 17 may 1490 20 march 1568 was the last grand master of the teutonic knights who a'
decodedNormWikiSample b' albert of prussia 17 may 1490 20 march 1568 was the last grand master of the teutonic knights who a'
wikiSample        b'@@1514 Albert of

In [64]:
save(encodedLena, "wyniki/lena.bin")
save(encodedNormWikiSample, "wyniki/norm_wiki_sample.bin")
save(encodedWikiSample, "wyniki/wiki_sample.bin")
loadedLena = load("wyniki/lena.bin")
loadedNormWikiSample = load("wyniki/norm_wiki_sample.bin")
loadedWikiSample = load("wyniki/wiki_sample.bin")

In [65]:
decodedLoadedLena = decode(loadedLena)
decodedLoadedNormWikiSample = decode(loadedNormWikiSample)
decodedLoadedWikiSample = decode(loadedWikiSample)
print("Czy zakodowany tekst jest taki sam jak oryginalny?", decodedLoadedLena == lena)
print("Czy zakodowany tekst jest taki sam jak oryginalny?", decodedLoadedNormWikiSample == normWikiSample)
print("Czy zakodowany tekst jest taki sam jak oryginalny?", decodedLoadedWikiSample == wikiSample)

Czy zakodowany tekst jest taki sam jak oryginalny? True
Czy zakodowany tekst jest taki sam jak oryginalny? True
Czy zakodowany tekst jest taki sam jak oryginalny? True
